# Abfrage open data aus Ratsinformationssystem

In [1]:
import pandas as pd
import requests
import json

In [2]:
v_ris_url = 'https://ris-oparl.itk-rheinland.de/Oparl/bodies/0015'

In [3]:
# get json for first level json-requeset to RIS

In [4]:
response = requests.get(v_ris_url)

In [5]:
d_response_json = response.json()

In [6]:
print(d_response_json)

{'id': 'http://ris-oparl.itk-rheinland.de/Oparl/bodies/0015', 'type': 'https://schema.oparl.org/1.0/Body', 'system': 'http://ris-oparl.itk-rheinland.de/Oparl/system', 'shortName': 'Landeshauptstadt Düsseldorf', 'name': 'LHD0015', 'licenseValidSince': '2017-01-31T00:00:00+02:00', 'oparlSince': '2017-01-31T00:00:00+02:00', 'equivalent': [''], 'organization': 'http://ris-oparl.itk-rheinland.de/Oparl/bodies/0015/organizations', 'person': 'http://ris-oparl.itk-rheinland.de/Oparl/bodies/0015/people', 'meeting': 'http://ris-oparl.itk-rheinland.de/Oparl/bodies/0015/meetings', 'paper': 'http://ris-oparl.itk-rheinland.de/Oparl/bodies/0015/papers', 'created': '2017-01-31T00:00:00+02:00', 'modified': '2017-01-31T00:00:00+02:00'}


In [7]:
l_response_keys = list(d_response_json.keys())

In [8]:
l_response_keys

['id',
 'type',
 'system',
 'shortName',
 'name',
 'licenseValidSince',
 'oparlSince',
 'equivalent',
 'organization',
 'person',
 'meeting',
 'paper',
 'created',
 'modified']

# Get json-response for most important keys

In [9]:
l_relevant_keys = l_response_keys[8:12]

In [10]:
l_relevant_keys # main pages with relevant information >> further info retrieve only necessary for these

['organization', 'person', 'meeting', 'paper']

In [11]:
# take these keys that access the actual information pages, loop requests
# to the corresponding URL (which is the value of the dict-entry) and write the
# result to a list containing 4 "response dictionaries"

# d_response_json.get('person')

In [12]:
l_response_dicts = []

In [13]:
for k in range(len(l_relevant_keys)):
    v_url = d_response_json.get(l_relevant_keys[k])
    response = requests.get(v_url)
    d_json = response.json()
    l_response_dicts.append(d_json)

# Create & export data frame for all papers in RIS

In [14]:
# l_response_dicts[3] # papers dictionary

In [15]:
### building a data frame from all papers with selected information
### Problem: no consistent data structure; not always all elements served (e.g. some records deleted)
### so data frame has to be build up line by line from response data with corresponding checks

In [16]:
########## initiate empty output data frame defining only the column names
l_keys_df_col_names = ['PAPER_REFERENCE', 'PAPER_NAME',  'PAPER_TYPE', 'DATE', 'DOWNLOAD_URL', 'ID']
df_papers = pd.DataFrame(columns = l_keys_df_col_names)
df_papers.head(5)

,PAPER_REFERENCE,PAPER_NAME,PAPER_TYPE,DATE,DOWNLOAD_URL,ID


In [17]:
### retrieving only the information on papers from the previous request list

In [18]:
l_papers_json = l_response_dicts[3].get('data') # retrieve data for "papers"-key (index 3 in json-repsonse-list)

In [23]:
# l_papers_json

In [20]:
for p in range(len(l_papers_json)): # loop over the number of papers found in the response object
    
    if l_papers_json[p].get('deleted') != None:   # check if this is a "deleted" and thus empty entry
        continue
        
    l_row_values =[]
    l_row_values.append(l_papers_json[p].get('reference'))
    l_row_values.append(l_papers_json[p].get('name'))
    l_row_values.append(l_papers_json[p].get('paperType'))
    l_row_values.append(l_papers_json[p].get('date'))
    
    l_aux_url = (l_papers_json[p].get('mainFile')) # accessUrl is packaged in seperate mainFile dictionary
    l_row_values.append(l_aux_url.get('downloadUrl'))

    l_row_values.append(l_papers_json[p].get('id'))
    
    # zipping key and value lists to dictionary and append as new row to df_out
    d_new_row = dict(zip(l_keys_df_col_names, l_row_values))
    # print(d_new_row)
    df_papers = df_papers.append(d_new_row, ignore_index = True)

In [21]:
df_papers.head(10)

,PAPER_REFERENCE,PAPER_NAME,PAPER_TYPE,DATE,DOWNLOAD_URL,ID
0,BV1/128/2020,Opernhaus Düsseldorf - Antrag der CDU-Fraktion,Antrag,2020-03-13,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,http://ris-oparl.itk-rheinland.de/Oparl/bodies...
1,BV1/127/2020,Aufstellflächen für Zulieferfirmen im Quartier...,Antrag,2020-03-13,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,http://ris-oparl.itk-rheinland.de/Oparl/bodies...
2,BV1/126/2020,Sonnenschutz im Hof der Jugendfreizeiteinricht...,Antrag,2020-03-13,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,http://ris-oparl.itk-rheinland.de/Oparl/bodies...
3,BV6/062/2020,"Bauunterhaltungsmittel 2020 - Teil B ""sonstige...",Bezirksvertretung Beschlussvorlage,2020-03-13,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,http://ris-oparl.itk-rheinland.de/Oparl/bodies...
4,BV6/061/2020,Verfügungsmittel 2020 - Schulgarten Friedrich-...,Bezirksvertretung Beschlussvorlage,2020-03-13,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,http://ris-oparl.itk-rheinland.de/Oparl/bodies...
5,BV1/124/2020,Status der Fernwärmebauarbeiten auf der Roßstr...,Anfrage,2020-03-13,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,http://ris-oparl.itk-rheinland.de/Oparl/bodies...
6,BV1/123/2020,Vermeidung von weiterer Verschmutzung der Grün...,Antrag,2020-03-13,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,http://ris-oparl.itk-rheinland.de/Oparl/bodies...
7,BV1/122/2020,Unterhaltung von Grundstücken und baulichen An...,Bezirksvertretung Beschlussvorlage,2020-03-16,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,http://ris-oparl.itk-rheinland.de/Oparl/bodies...


In [ ]:
## write result to Excel file

In [22]:
writer = pd.ExcelWriter('_Dokumente_Ratsinformationssystem.xlsx', engine='xlsxwriter')
df_papers.to_excel(writer, sheet_name='papers', engine='xlsxwriter')
writer.save()